<a href="https://colab.research.google.com/github/dangduong2806/google-colab/blob/main/Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving colors1.csv to colors1.csv


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import pandas as pd
import transformers
from tensorflow.keras.layers import Input, Dense, Lambda

In [3]:
df = pd.read_csv("colors1.csv")
df.head()

,name,red,green,blue
0,parakeet,174,182,87
1,saddle brown,88,52,1
2,cucumber crush,222,237,215
3,pool blue,134,194,201
4,distance,98,110,130


Standardize data

In [4]:
df["red"] = df["red"] / 255
df["green"] = df["green"] / 255
df["blue"] = df["blue"] / 255
df.head()

,name,red,green,blue
0,parakeet,0.682353,0.713725,0.341176
1,saddle brown,0.345098,0.203922,0.003922
2,cucumber crush,0.870588,0.929412,0.843137
3,pool blue,0.525490,0.760784,0.788235
4,distance,0.384314,0.431373,0.509804


In [5]:
len(df["name"])

14357

Tách input và output

In [6]:
X_texts = df['name'].astype(str).values
y_rgb = df[['red', 'green', 'blue']].values

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# Tách train/val
X_train, X_val, y_train, y_val = train_test_split(
    X_texts, y_rgb, test_size=0.2, random_state=42
)

In [9]:
from transformers import BertTokenizer


In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_text(texts):
    texts = list(texts)
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
    return {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "token_type_ids": encodings["token_type_ids"]
    }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
from transformers import TFBertModel

In [12]:
def create_bert_to_rgb_model():
    bert = TFBertModel.from_pretrained("bert-base-uncased")
    for layer in bert.layers:
        layer.trainable = False  # freeze toàn bộ BERT

    input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

    # Combined Lambda layer that does both BERT call and CLS extraction
    cls_token = tf.keras.layers.Lambda(
        lambda x: bert(input_ids=x[0], attention_mask=x[1]).last_hidden_state[:, 0, :],
        output_shape=(768,)
    )([input_ids, attention_mask])

    # Chỉ train lớp này
    x = tf.keras.layers.Dense(128, activation="relu")(cls_token)
    x = tf.keras.layers.Dense(3, activation="sigmoid")(x)  # sigmoid vì RGB chuẩn hóa [0, 1]

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=x)
    return model

In [13]:
model = create_bert_to_rgb_model()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="mse",
    metrics=["mae", "mse"]
)

In [15]:
train_tokens = tokenize_text(X_train)
val_tokens = tokenize_text(X_val)

In [16]:
history = model.fit(
    train_tokens,
    y_train,
    validation_data=(val_tokens, y_val),
    epochs=30,
    batch_size=16,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)


Epoch 1/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - loss: 0.0634 - mae: 0.2109 - mse: 0.0634 - val_loss: 0.0543 - val_mae: 0.1953 - val_mse: 0.0543
Epoch 2/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.0533 - mae: 0.1921 - mse: 0.0533 - val_loss: 0.0513 - val_mae: 0.1877 - val_mse: 0.0513
Epoch 3/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0504 - mae: 0.1861 - mse: 0.0504 - val_loss: 0.0499 - val_mae: 0.1857 - val_mse: 0.0499
Epoch 4/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0479 - mae: 0.1803 - mse: 0.0479 - val_loss: 0.0493 - val_mae: 0.1842 - val_mse: 0.0493
Epoch 5/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - loss: 0.0474 - mae: 0.1788 - mse: 0.0474 - val_loss: 0.0490 - val_mae: 0.1832 - val_mse: 0.0490
Epoch 6/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - loss: 0.0461 - mae: 0.1757 - mse: 0.0461 - val_loss: 0.0490 - val_mae: 0.1828 - val_mse: 0.0490
Epoch 7/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 0.0456 - mae: 0.1747 - mse